### A dirty little trick to avoid having to deal with the notion of relative path. 

In [1]:
import os
os.chdir('/home/juagudelo/HOMEdev/drought_pred_hybrid/')

In [ ]:
os.getcwd()

'/home/juagudelo/HOMEdev/drought_pred_hybrid'

## From here, this notebook is going to be used to evaluate the best model issued from the hyperparameter tuning over the test set.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import pickle
from tqdm import tqdm
from scipy.interpolate import interp1d
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import f1_score, mean_absolute_error

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

import models
import utilities

In [4]:
# Fixing a seed to warrant the reproducibility
torch.manual_seed(21)
np.random.seed(21)

In [5]:
# Initializing the model
num_categorical_features=7
num_numerical_features=22
num_time_series_features=21
batch_size=128
output_weeks=6
# Hyperparameters
hidden_size= 340
num_lstm_layers= 8
embedding_dims= 270
num_fc_tabular_layers= 4
num_fc_combined_layers= 1
dropout= 0.30000000000000004

In [6]:
# Importing the list of unique categories for the categorical features
with open(f"data/processed_dataFrames/list_cat.pickle", "rb") as f:
    list_cat = pickle.load(f)

In [7]:
# Setting up the loaders
dfs=utilities.load_dataFrames()
valid_loader=utilities.create_dataLoader(X_static=dfs["X_tabular_valid"],
                                         X_static_cat=dfs["X_tabular_cat_valid"],
                                         X_time=dfs["X_time_valid"],
                                         y_target=dfs["y_target_valid"],
                                         output_weeks=output_weeks,
                                         y_past=None,
                                         batch_size=128,
                                         shuffle=False
                                         )
test_loader=utilities.create_dataLoader(X_static=dfs["X_tabular_test"],
                                        X_static_cat=dfs["X_tabular_cat_test"],
                                        X_time=dfs["X_time_test"],
                                        y_target=dfs["y_target_test"],
                                        output_weeks=output_weeks,
                                        y_past=None,
                                        batch_size=128,
                                        shuffle=False
                                        )

In [8]:
# set up the device
device=torch.device("cpu")
print(f"Using device: {device}")
print(torch.cuda.get_device_name(device=None))

model = models.HybridModel(num_categorical_features,
                           list_cat,
                           num_numerical_features,
                           num_time_series_features,
                           hidden_size,
                           num_lstm_layers,
                           dropout,
                           embedding_dims,
                           num_fc_tabular_layers,
                           num_fc_combined_layers,
                           output_size=output_weeks,
)

model.load_state_dict(torch.load("models/MH_Hyper/MH_Hyper_12.pt", weights_only=True))
model.to(device)

Using device: cpu
NVIDIA T1000 8GB


HybridModel(
  (embeddings): ModuleList(
    (0-4): 5 x Embedding(7, 270)
    (5): Embedding(6, 270)
    (6): Embedding(8, 270)
  )
  (tabular_fc_layers): Sequential(
    (0): Linear(in_features=1912, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=128, bias=True)
    (7): ReLU()
    (8): Linear(in_features=128, out_features=64, bias=True)
    (9): ReLU()
  )
  (lstm): LSTM(21, 340, num_layers=8, batch_first=True)
  (attention): Linear(in_features=340, out_features=1, bias=True)
  (dropout): Dropout(p=0.30000000000000004, inplace=False)
  (fc_after_context): Linear(in_features=340, out_features=64, bias=True)
  (combined_fc_layers): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): Re

In [9]:
def predict(x, static, static_cat):
    out = model(static_cat, static, x)
    return out

In [10]:
dict_map = {
    "y_pred": [],
    "y_pred_rounded": [],
    # "fips": [],
    # "date": [],
    "y_true": [],
    "week": [],
}
i = 0
for static, static_cat, x, y in tqdm(
    test_loader,
    desc="Test predictions...",
):
    x, static, y = x.to(device), static.to(device), y.to(device)
    with torch.no_grad():
        pred = predict(x, static, static_cat).clone().detach()
    for w in range(output_weeks):
        dict_map["y_pred"] += [float(p[w]) for p in pred]
        dict_map["y_pred_rounded"] += [int(p.round()[w]) for p in pred]
        # dict_map["fips"] += [f[1][0] for f in valid_fips[i : i + len(x)]]
        # dict_map["date"] += [f[1][1] for f in valid_fips[i : i + len(x)]]
        dict_map["y_true"] += [float(item[w]) for item in y]
        dict_map["week"] += [w] * len(x)
    i += len(x)
df = pd.DataFrame(dict_map)

Test predictions...: 100%|██████████| 69/69 [00:43<00:00,  1.58it/s]


In [11]:
df

,y_pred,y_pred_rounded,y_true,week
0,2.056026,2,2.8891,0
1,0.329628,0,0.8519,0
2,0.090850,0,0.0000,0
3,1.557501,2,1.4617,0
4,0.936767,1,1.6942,0
...,...,...,...,...
52603,1.076862,1,0.0000,5
52604,1.719404,2,2.0000,5
52605,0.186367,0,0.0000,5
52606,0.218914,0,0.0000,5


In [12]:
for w in range(6):
    wdf = df[df['week']==w]
    mae = mean_absolute_error(wdf['y_true'], wdf['y_pred']).round(3)
    f1 = f1_score(wdf['y_true'].round(),wdf['y_pred'].round(), average='macro').round(3)
    print(f"Week {w+1}", f"MAE {mae}", f"F1 {f1}")


Week 1 MAE 0.131 F1 0.758
Week 2 MAE 0.198 F1 0.647
Week 3 MAE 0.239 F1 0.595
Week 4 MAE 0.291 F1 0.559
Week 5 MAE 0.33 F1 0.514
Week 6 MAE 0.369 F1 0.476
